# (5.1) Solve AFAPE for dataset completed with multiple imputation
Use an estimator to evaluate E[C|do(R_bar = 1)]. Also give valid confidence intervals (through estimating mean and variance). 

In [14]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Define paths

Paths for data

In [15]:
from afa.configurations.utils_ts import specify_default_paths_ts
# which dataset to work on 
dataset_name   = "synthetic_4_0"

# name for of missingness scenario 
miss_scenario  = 'MCAR_1'

# automatically specify some path locations (change paths manually if needed) 
paths = specify_default_paths_ts(dataset_name = dataset_name , miss_scenario = miss_scenario) 

# name for agent 
agent_name            = 'DQN'
agent_name            = 'Random 0% with TCN'

agent_dir = paths['data_dir'] + 'afa_agents' + '/' + agent_name + '/'

# how to name the afa_dataset
mi_model_name   =  'mi_simple'
mi_model_dir  =  paths['data_dir'] + 'mi_models/' + mi_model_name + '/'
afa_dataset_name = mi_model_name 

## Load afa dataset

In [3]:
from afa.data_modelling.datasets.data_loader.data_loader_ts import DataLoader_ts
from afa.data_modelling.missingness.multiple_imputation.multiple_imputed_data_loader.multiple_imputed_data_loader_ts import MultipleImputedDataLoader_ts
from afa.afa_datasets.afa_data_loader.afa_data_loader_ts import AFADataLoader_ts

2023-05-11 12:32:38.197983: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-11 12:32:38.354816: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-05-11 12:32:38.354836: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-05-11 12:32:39.069837: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [4]:
# load dataset 
data_loader = DataLoader_ts(     data_file                  = paths['data_file'],
                                 temporal_data_file         = paths['temporal_data_file'],
                                 superfeature_mapping_file  = paths['superfeature_mapping_file'],
                                 problem_file               = paths['problem_file'],
                                 afa_problem_files          = paths['afa_problem_files'],
                                 folds_file                 = paths['folds_file'] )
dataset = data_loader.load() 

Padding sequences: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 1767.18it/s]


In [5]:
# load mi dataset
augmented_data_file = mi_model_dir + 'results.hkl'
mi_model_params = None
mi_data_loader = MultipleImputedDataLoader_ts(                   
                    augmented_data_file = augmented_data_file,
                    dataset             = dataset,
                    model_params        = mi_model_params) 
mi_dataset = mi_data_loader.load() 

In [6]:
mi_dataset.results.keys()

dict_keys(['ids', 'target_feature'])

In [7]:
#load afa_dataset
augmented_data_file = agent_dir + afa_dataset_name + '_' + 'results.hkl'
afa_agent_params = None
afa_data_loader = AFADataLoader_ts(                   
                    augmented_data_file = augmented_data_file,
                    dataset             = mi_dataset,
                    model_params        = afa_agent_params) 
afa_dataset = afa_data_loader.load() 

2023-05-11 12:32:50.104957: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-11 12:32:50.105302: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcufft.so.10'; dlerror: libcufft.so.10: cannot open shared object file: No such file or directory
2023-05-11 12:32:50.105395: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcurand.so.10'; dlerror: libcurand.so.10: cannot open shared object file: No such file or directory
2023-05-11 12:32:50.105465: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcusolver.so.11'; dlerror: libcusolver.so.11: cannot open shared object file: No such file or directory
2023-05-11 12:32:50.105524: W tensorflow/co

In [8]:
data, results = afa_dataset.get_augmented_data()
results.keys()

dict_keys(['ids', 'target_superR_hat', 'label_pred', 'cost_acq', 'cost_mc', 'cost', 'mi_number'])

In [13]:
data['temporal_feature']

array([[[ 1.86501492, -0.60501799, -0.0148572 ,  0.01026182],
        [-0.04280796, -0.12856489, -0.43601789, -1.14884891],
        [-0.53436443,  1.22542717, -0.78219913, -0.03030623],
        [-0.6894841 , -0.54329778, -0.81760127,  0.81351747],
        [-0.50308986,  0.66748985, -0.71051529, -0.641935  ]],

       [[ 1.86501492, -0.60501799, -0.0148572 ,  0.01026182],
        [-0.04280796, -0.12856489, -0.43601789, -1.14884891],
        [-0.53436443,  1.22542717, -0.78219913, -0.03030623],
        [-0.6894841 , -0.54329778, -0.81760127,  0.81351747],
        [-0.50308986,  0.66748985, -0.71051529, -0.641935  ]],

       [[ 1.86501492, -0.60501799, -0.0148572 ,  0.01026182],
        [-0.04280796, -0.12856489, -0.43601789, -1.14884891],
        [-0.53436443,  1.22542717, -0.78219913, -0.03030623],
        [-0.6894841 , -0.54329778, -0.81760127,  0.81351747],
        [-0.50308986,  0.66748985, -0.71051529, -0.641935  ]],

       ...,

       [[ 0.34686034,  0.89982627, -0.50553155,  1.

In [11]:
results['cost_mc'].shape

(500, 5)

## Compute estimates 

In [ ]:
J_bootstraps = afa_dataset.estimate_counterfactual_cost_mi(  mi_model_name  = mi_model_name , 
                                                             fold = 0, split = "test", 
                                                             n_samples = None, 
                                                             n_bootstraps = 10)

In [ ]:
# save estimate
from afa.afa_models.afa_estimators.utils import save_results_bootstrapping
save_results_bootstrapping( J_bootstraps , agent_dir, afa_dataset_name = afa_dataset_name )

## Compute estimates for convergence
If we know the ground truth, we might be interesting in plotting convergence, for this we might want to compute estimates J for different amount of available datapoints. 

In [ ]:
from afa.afa_models.afa_estimators.utils_ts import define_afa_estimator_ts
from afa.afa_models.afa_estimators.utils import compute_counterfactual_cost_convergence_mi

In [ ]:
# init estiamtor 
estimators = [define_afa_estimator_ts(    estimator_name   = mi_model_name ,
                                              estimator_type   = 'simple_blocking' ,
                                              estimator_params = None) ]

In [ ]:
# define estimators 
J_bootstraps_convergence, convergence_steps  = compute_counterfactual_cost_convergence_mi(   afa_dataset = afa_dataset, 
                                                                                             mi_model_name = mi_model_name, 
                                                                                             fold = 0, split = "test", 
                                                                                             n_samples = 1, 
                                                                                             n_bootstraps = 10)

In [ ]:
# save estimators
save_results_bootstrapping( J_bootstraps_convergence , agent_dir, convergence_steps = convergence_steps, afa_dataset_name = afa_dataset_name)